In [1]:
from data import Database
import pandas as pd

In [2]:
db = Database()
df = db.dataframe()

In [3]:
df

,Name,Type,Level,Rarity,Damage,Health,Energy,Sanity,Timestamp
_id,,,,,,,,,
655d631bca59d65c5710e0e6,Red Wyrmling,Dragon,11,Rank 2,11d6+2,67.38,64.28,64.96,2023-11-21 18:10:35
655d631bca59d65c5710e0bd,Death Knight,Undead,11,Rank 1,11d4+1,42.33,42.57,43.56,2023-11-21 18:10:35
655d631bca59d65c5710e0ba,Dust Spirit,Fey,8,Rank 5,8d12+2,94.39,96.10,96.23,2023-11-21 18:10:35
655d631bca59d65c5710e0be,Goblin Guard,Devilkin,5,Rank 1,5d4,18.83,18.10,18.73,2023-11-21 18:10:35
655d631bca59d65c5710e0b9,Smoke Mephit,Elemental,13,Rank 3,13d8,100.87,100.31,103.28,2023-11-21 18:10:35
...,...,...,...,...,...,...,...,...,...
655d631bca59d65c5710e46e,Diamond Wyrmling,Dragon,12,Rank 4,12d10+1,124.76,123.70,121.63,2023-11-21 18:10:35
655d631bca59d65c5710e495,Brass Faerie,Fey,14,Rank 0,14d2+3,28.93,28.43,28.23,2023-11-21 18:10:35
655d631bca59d65c5710e4ac,Poltergeist,Undead,6,Rank 3,6d8,45.11,44.84,50.85,2023-11-21 18:10:35


In [87]:
X = df[['Level', 'Health', 'Energy', 'Sanity']]
y = df['Rarity']

In [88]:
from sklearn.model_selection import train_test_split

X_train, X_test, y_train, y_test = train_test_split(X, y)

In [89]:
df['Rarity'].value_counts(normalize=True)

Rarity
Rank 0    0.300781
Rank 1    0.230469
Rank 2    0.209961
Rank 3    0.143555
Rank 4    0.090820
Rank 5    0.024414
Name: proportion, dtype: float64

In [90]:
from sklearn.dummy import DummyClassifier

dummy = DummyClassifier(strategy='most_frequent')

dummy.fit(X_train, y_train)

dummy.score(X_test, y_test)

0.265625

In [106]:
from sklearn.linear_model import LogisticRegression

lr = LogisticRegression(max_iter=1000)

lr.fit(X_train, y_train)

lr.score(X_test, y_test)

0.953125

We are not using the StandardScaler in our data preprocessing pipeline because the "Level" feature in our dataset is not normally distributed. The StandardScaler assumes that the data follows a normal distribution, but in our case, the distribution of the "Level" feature deviates from normality. Applying the StandardScaler degrades performance of the lr model (I verified this empirically). Our features (and therefore coefficients) are still similar enough in scale that this is not a serious concern.

In [108]:
df['Level'].value_counts(normalize=True).sort_index()

Level
1     0.027344
2     0.096680
3     0.097656
4     0.073242
5     0.080078
6     0.092773
7     0.070312
8     0.075195
9     0.048828
10    0.060547
11    0.058594
12    0.054688
13    0.046875
14    0.033203
15    0.026367
16    0.026367
17    0.014648
18    0.007812
19    0.006836
20    0.001953
Name: proportion, dtype: float64

In [93]:
lr.coef_

array([[ 7.43626404, -0.62365711, -0.59467101, -0.67358305],
       [ 3.2336191 , -0.0549516 , -0.25553982, -0.05812833],
       [ 0.94798661, -0.0244049 ,  0.07795233,  0.04566017],
       [-1.39813478,  0.09397416,  0.24226475,  0.10574671],
       [-4.79061272,  0.28073955,  0.32056558,  0.23580602],
       [-5.42912225,  0.3282999 ,  0.20942817,  0.34449848]])

In [94]:
from sklearn.neighbors import KNeighborsClassifier

knn = KNeighborsClassifier()

knn.fit(X_train, y_train)

knn.score(X_test, y_test)

0.796875

In [95]:
from sklearn.tree import DecisionTreeClassifier

dt = DecisionTreeClassifier()

dt.fit(X_train, y_train)

dt.score(X_test, y_test)

0.91015625

In [96]:
from sklearn.ensemble import RandomForestClassifier

rf = RandomForestClassifier()

rf.fit(X_train, y_train)

rf.score(X_test, y_test)

0.94140625

In [107]:
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import KFold, cross_val_score

kf = KFold(n_splits=5, shuffle=True, random_state=1948)

cross_val_score(lr, X, y, cv=kf, n_jobs=-1)

/Users/gregorysamek/anaconda3/envs/bandersnatchstarter/lib/python3.12/site-packages/sklearn/linear_model/_logistic.py:460: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(


array([0.97560976, 0.96097561, 0.9804878 , 0.94634146, 0.9754902 ])

In [98]:
cross_val_score(rf, X, y, cv=kf, n_jobs=-1)

array([0.95609756, 0.94146341, 0.96585366, 0.94146341, 0.94607843])

Logisitic Regression appears to perform better than even a random forest classifier. Given that our data was generated synthetically using a straightforward process, it makes sense that regression is sufficient to model the data well. Given the simplicity (and therefore speed) of regression relative to a more complex ensemble classifier, lr is the obvious choice for our production model.